In [1]:
!nvidia-smi

Mon Jan 26 00:58:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   28C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip -q install -U "transformers>=4.41" "datasets>=2.20" "accelerate>=0.33" peft deepspeed sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.3 MB/s eta 0:00:00


In [3]:
%%bash
cat > ds_zero2_bf16.json <<'JSON'
{
  "bf16": { "enabled": true },
  "zero_optimization": {
    "stage": 2,
    "allgather_partitions": true,
    "allgather_bucket_size": 200000000,
    "reduce_scatter": true,
    "reduce_bucket_size": 200000000,
    "overlap_comm": true,
    "contiguous_gradients": true
  },
  "gradient_accumulation_steps": 16,
  "train_micro_batch_size_per_gpu": 1,
  "wall_clock_breakdown": false
}
JSON


In [17]:
%%bash
cat > sft_lora_ds_hf.py <<'PY'
import argparse
from typing import Dict, Any, List, Optional
from inspect import signature

import torch
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model


def parse_args():
    p = argparse.ArgumentParser()
    # DeepSpeed injects this
    p.add_argument("--local_rank", type=int, default=-1)

    p.add_argument("--model_name_or_path", type=str, required=True)
    p.add_argument("--output_dir", type=str, required=True)

    p.add_argument("--dataset_1", type=str, required=True)
    p.add_argument("--dataset_1_split", type=str, default="train")
    p.add_argument("--dataset_1_n", type=int, default=2000)

    p.add_argument("--dataset_2", type=str, default=None)
    p.add_argument("--dataset_2_split", type=str, default="train")
    p.add_argument("--dataset_2_n", type=int, default=600)

    p.add_argument("--text_field", type=str, default=None)

    p.add_argument("--num_train_epochs", type=float, default=1)
    p.add_argument("--per_device_train_batch_size", type=int, default=1)
    p.add_argument("--per_device_eval_batch_size", type=int, default=1)
    p.add_argument("--gradient_accumulation_steps", type=int, default=16)
    p.add_argument("--learning_rate", type=float, default=1e-4)
    p.add_argument("--warmup_ratio", type=float, default=0.05)
    p.add_argument("--logging_steps", type=int, default=5)
    p.add_argument("--eval_steps", type=int, default=50)
    p.add_argument("--save_steps", type=int, default=50)
    p.add_argument("--save_total_limit", type=int, default=2)
    p.add_argument("--max_seq_length", type=int, default=2048)
    p.add_argument("--dataloader_num_workers", type=int, default=4)

    p.add_argument("--deepspeed", type=str, default=None)
    p.add_argument("--bf16", type=str, default="True")

    p.add_argument("--lora_r", type=int, default=8)
    p.add_argument("--lora_alpha", type=int, default=32)
    p.add_argument("--lora_target_modules", type=str, default="all-linear")
    return p.parse_args()


def pick_all_linear_target_modules(model) -> List[str]:
    names = set()
    for name, module in model.named_modules():
        if module.__class__.__name__ in ("Linear", "Linear4bit", "Linear8bitLt"):
            names.add(name.split(".")[-1])
    return sorted(list(names))


def format_example(ex: Dict[str, Any], tokenizer, text_field: Optional[str]) -> str:
    if text_field and text_field in ex and ex[text_field]:
        return ex[text_field]

    if "instruction" in ex and "output" in ex:
        inp = ex.get("input", "")
        if inp:
            return f"### Instruction:\n{ex['instruction']}\n\n### Input:\n{inp}\n\n### Response:\n{ex['output']}"
        return f"### Instruction:\n{ex['instruction']}\n\n### Response:\n{ex['output']}"

    if "prompt" in ex and ("completion" in ex or "response" in ex):
        comp = ex.get("completion", ex.get("response", ""))
        return f"{ex['prompt']}{comp}"

    if "question" in ex and "answer" in ex:
        return f"### Question:\n{ex['question']}\n\n### Answer:\n{ex['answer']}"

    if "messages" in ex and ex["messages"]:
        try:
            return tokenizer.apply_chat_template(ex["messages"], tokenize=False, add_generation_prompt=False)
        except Exception:
            return str(ex["messages"])

    if "conversations" in ex and ex["conversations"]:
        try:
            msgs = []
            for m in ex["conversations"]:
                role = m.get("from", "")
                content = m.get("value", "")
                if role in ("human", "user"):
                    msgs.append({"role": "user", "content": content})
                else:
                    msgs.append({"role": "assistant", "content": content})
            return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
        except Exception:
            return str(ex["conversations"])

    return str(ex)


def load_n(name: str, split: str, n: int) -> Dataset:
    ds = load_dataset(name, split=split)
    if n and n < len(ds):
        ds = ds.select(range(n))
    return ds


def build_training_args(args, bf16: bool) -> TrainingArguments:
    # Build kwargs compatible with the transformers version installed
    base = dict(
        output_dir=args.output_dir,
        num_train_epochs=args.num_train_epochs,
        per_device_train_batch_size=args.per_device_train_batch_size,
        per_device_eval_batch_size=args.per_device_eval_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        learning_rate=args.learning_rate,
        warmup_ratio=args.warmup_ratio,
        logging_steps=args.logging_steps,
        eval_steps=args.eval_steps,
        save_steps=args.save_steps,
        save_total_limit=args.save_total_limit,
        bf16=bf16,
        dataloader_num_workers=args.dataloader_num_workers,
        deepspeed=args.deepspeed,
        report_to="none",
    )

    params = signature(TrainingArguments.__init__).parameters
    if "evaluation_strategy" in params:
        base["evaluation_strategy"] = "steps"
    elif "eval_strategy" in params:
        base["eval_strategy"] = "steps"

    if "save_strategy" in params:
        base["save_strategy"] = "steps"

    return TrainingArguments(**base)


def main():
    args = parse_args()
    bf16 = args.bf16.lower() == "true"

    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        torch_dtype=torch.bfloat16 if bf16 else torch.float16,
        trust_remote_code=True,
    )

    if args.lora_target_modules == "all-linear":
        target_modules = pick_all_linear_target_modules(model)
    else:
        target_modules = [x.strip() for x in args.lora_target_modules.split(",") if x.strip()]

    lora_cfg = LoraConfig(
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        target_modules=target_modules,
        lora_dropout=0.0,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, lora_cfg)

    ds1 = load_n(args.dataset_1, args.dataset_1_split, args.dataset_1_n)
    if args.dataset_2:
        ds2 = load_n(args.dataset_2, args.dataset_2_split, args.dataset_2_n)
        train_ds = concatenate_datasets([ds1, ds2])
    else:
        train_ds = ds1

    def tok_fn(ex):
        text = format_example(ex, tokenizer, args.text_field)
        return tokenizer(text, truncation=True, max_length=args.max_seq_length, padding=False)

    train_ds = train_ds.map(tok_fn, remove_columns=train_ds.column_names)
    eval_ds = train_ds.select(range(min(64, len(train_ds))))

    collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    targs = build_training_args(args, bf16)

    trainer = Trainer(
        model=model,
        args=targs,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        data_collator=collator,
    )

    trainer.train()
    trainer.save_model(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)


if __name__ == "__main__":
    main()
PY


### Run training with DeepSpeed (single GPU)

In [19]:
!deepspeed --num_gpus=1 sft_lora_ds_hf.py \
  --model_name_or_path Qwen/Qwen3-0.6B \
  --output_dir output \
  --num_train_epochs 1 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1 \
  --gradient_accumulation_steps 16 \
  --learning_rate 1e-4 \
  --warmup_ratio 0.05 \
  --logging_steps 5 \
  --eval_steps 50 \
  --save_steps 50 \
  --save_total_limit 2 \
  --max_seq_length 2048 \
  --dataloader_num_workers 4 \
  --bf16 True \
  --deepspeed ds_zero2_bf16.json \
  --lora_r 8 \
  --lora_alpha 32 \
  --lora_target_modules all-linear \
  --dataset_1 tatsu-lab/alpaca --dataset_1_split train --dataset_1_n 2000

2026-01-26 01:13:36.566552: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 01:13:36.583988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769390016.605237    6559 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769390016.611837    6559 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769390016.628101    6559 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [20]:
!ls -lah output

total 324M
drwxr-xr-x 4 root root 4.0K Jan 26 01:23 .
drwxr-xr-x 1 root root 4.0K Jan 26 01:14 ..
-rw-r--r-- 1 root root 1.1K Jan 26 01:23 adapter_config.json
-rw-r--r-- 1 root root 309M Jan 26 01:23 adapter_model.safetensors
-rw-r--r-- 1 root root  707 Jan 26 01:23 added_tokens.json
-rw-r--r-- 1 root root 4.1K Jan 26 01:23 chat_template.jinja
drwxr-xr-x 3 root root 4.0K Jan 26 01:21 checkpoint-100
drwxr-xr-x 3 root root 4.0K Jan 26 01:23 checkpoint-125
-rw-r--r-- 1 root root 1.6M Jan 26 01:23 merges.txt
-rw-r--r-- 1 root root 5.1K Jan 26 01:23 README.md
-rw-r--r-- 1 root root  613 Jan 26 01:23 special_tokens_map.json
-rw-r--r-- 1 root root 5.3K Jan 26 01:23 tokenizer_config.json
-rw-r--r-- 1 root root  11M Jan 26 01:23 tokenizer.json
-rw-r--r-- 1 root root 7.0K Jan 26 01:23 training_args.bin
-rw-r--r-- 1 root root 2.7M Jan 26 01:23 vocab.json


In [21]:
!pip -q install -U transformers peft accelerate

### Inference

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE = "Qwen/Qwen3-0.6B"
ADAPTER_DIR = "output"   # your training output folder

tokenizer = AutoTokenizer.from_pretrained(BASE, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    BASE,
    torch_dtype=torch.float16,   # use fp16 for Colab T4; use bfloat16 on A100/L4 if supported
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, ADAPTER_DIR)
model.eval()

prompt = "### Instruction:\nExplain what LoRA is in 3 bullet points.\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:919: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


### Instruction:
Explain what LoRA is in 3 bullet points.

### Response:
LoRA stands for LoRA Optimization. It is an optimization technique used to adjust the weights of a neural network to improve its performance on a given task. It works by modifying the weights of the neural network to better fit the task, allowing it to make more accurate predictions. LoRA can be used to improve the performance of neural networks on tasks such as classification and regression, as well as to improve the performance of deep learning models. It is a fast and effective way to optimize deep learning models and can be used in any number of applications. It is particularly useful when the data is large and the task is complex, as it allows for faster and more accurate results. LoRA can be used in any number of applications, including computer vision, natural language processing, and robotics. It is a powerful and efficient way to improve the performance of neural networks. It can be used in any number of 

### Merge LoRA into the base model

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE = "Qwen/Qwen3-0.6B"
ADAPTER_DIR = "output"
MERGED_DIR = "output_merged"

tokenizer = AutoTokenizer.from_pretrained(BASE, trust_remote_code=True)

base = AutoModelForCausalLM.from_pretrained(
    BASE,
    torch_dtype=torch.float16,
    device_map="cpu",  # merge on CPU to save GPU memory; you can use "auto" if you have room
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(base, ADAPTER_DIR)
merged = peft_model.merge_and_unload()  # merges LoRA weights into base weights

merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("Saved merged model to:", MERGED_DIR)


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:572: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications. You can opt to merge the adapter after cloning the weights (to untie the embeddings). You can untie the embeddings by loading the model with `tie_word_embeddings=False`. For example:
```python
from transformers import AutoModelForCausalLM

# Load original tied model
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", tie_word_embeddings=False)

# Set the randomly initialized lm_head to the previously tied embeddings
model.lm_head.weight.data = model.model.embed_tokens.weight.data.clone()

# Save the untied model
untied_model_dir = "dir/for/untied/model"
model.save_pretrained(untied_model_dir)
model.config.save_pretrained(untied_model_dir)

# Now use the original model but in untied format
model = AutoModelForCausalLM.from_pretrained(untied_model_dir)
`

Saved merged model to: output_merged


### Inference with merged model

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MERGED_DIR = "output_merged"

tokenizer = AutoTokenizer.from_pretrained(MERGED_DIR, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

prompt = "Write a short email asking for a meeting time next week."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

out = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(out[0], skip_special_tokens=True))


The tokenizer you are loading from 'output_merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Write a short email asking for a meeting time next week. 

### Response:
Hi [First Name], 

I'd love to hear from you next week. Could you please tell me the time you'd like to meet? I'd be happy to help you out! 

Best regards, 
[Your Name] 

### Response:
Hi [First Name], 

I'd love to hear from you next week. Could you please tell me the time you'd like to meet? I'd be happy to help you out! 

Best regards, 
[Your Name] 

### Response:
Hi [First Name], 

I'd love to hear from you next week. Could you please tell me the time you'd like to meet? I'd be happy to help you out! 

Best regards, 
[
